In [14]:
# Process all Excel files in the directory
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path

# Define the directory path
directory_path = r"C:\Users\thaim\Downloads\בדיקת פלאש\20-08\ניסוי 2\הזזתי בטעות"

print("Processing all Excel files in directory...")
print(f"Directory: {directory_path}")

# Check if directory exists
if not os.path.exists(directory_path):
    print("Directory not found! Please check the path.")
else:
    # Find all CSV files in the directory
    csv_files = glob.glob(os.path.join(directory_path, "*.csv"))
    
    print(f"Found {len(csv_files)} CSV files:")
    for file in csv_files:
        print(f"  - {os.path.basename(file)}")
    
    if len(csv_files) == 0:
        print("No CSV files found in the directory!")
    else:
        print("\nStarting to process CSV files...")
        
        # Process each CSV file
        for file_idx, file_path in enumerate(csv_files):
            try:
                print(f"\n{'='*80}")
                print(f"Processing file {file_idx + 1}/{len(csv_files)}: {os.path.basename(file_path)}")
                print(f"{'='*80}")
                
                # Read the CSV file
                try:
                    # Try reading with different encodings
                    df = pd.read_csv(file_path, encoding='utf-8')
                except UnicodeDecodeError:
                    try:
                        df = pd.read_csv(file_path, encoding='windows-1255')  # Hebrew encoding
                    except UnicodeDecodeError:
                        try:
                            df = pd.read_csv(file_path, encoding='iso-8859-8')  # Another Hebrew encoding
                        except UnicodeDecodeError:
                            df = pd.read_csv(file_path, encoding='latin-1')  # Fallback encoding
                
                print(f"File loaded successfully!")
                print(f"Original shape: {df.shape}")
                
                # Display basic info about the file structure
                print(f"Columns: {df.columns.tolist()}")
                
                # Assuming first row is titles, second row is sensor names, data starts from row 3
                if len(df) > 2:
                    print("File structure detected:")
                    print(f"  Row 1: {df.iloc[0].tolist()}")
                    print(f"  Row 2: {df.iloc[1].tolist()}")
                    print(f"  Data starts from row 3")
                    print(f"  Total data rows: {len(df) - 2}")
                else:
                    print("Warning: File has less than 3 rows, skipping...")
                    continue
                
                # Process the current file - add max columns for every 500 rows
                try:
                    # Create a copy for processing
                    df_processed = df.copy()
                    
                    # Get data columns (assuming all columns contain sensor data)
                    data_columns = df.columns
                    
                    # Calculate intervals
                    data_rows = len(df) - 2  # Subtract 2 for header rows
                    num_intervals = data_rows // 500
                    remaining_rows = data_rows % 500
                    total_intervals = num_intervals + (1 if remaining_rows > 0 else 0)
                    
                    print(f"  Data analysis:")
                    print(f"    Total data rows: {data_rows}")
                    print(f"    Complete 500-row intervals: {num_intervals}")
                    if remaining_rows > 0:
                        print(f"    Remaining rows in last interval: {remaining_rows}")
                    print(f"    Total intervals: {total_intervals}")
                    
                    # Create new max columns
                    new_max_columns = {}
                    
                    for col in data_columns:
                        print(f"    Processing column: {col}")
                        
                        # Create new column name for max values
                        new_col_name = f"{col}_Max500"
                        new_max_columns[new_col_name] = []
                        
                        # Copy header rows first
                        new_max_columns[new_col_name].extend([df.iloc[0][col], df.iloc[1][col]])
                        
                        # Process each 500-row interval
                        max_values_for_avg = []  # Store max values for average calculation
                        
                        for interval in range(num_intervals):
                            start_row = 2 + (interval * 500)  # Start from row 3 (index 2)
                            end_row = start_row + 500
                            
                            # Extract the 500 rows for this interval
                            interval_data = df.iloc[start_row:end_row][col]
                            
                            # Convert to numeric and handle non-numeric values
                            numeric_data = pd.to_numeric(interval_data, errors='coerce')
                            clean_data = numeric_data.dropna()
                            
                            if len(clean_data) > 0:
                                interval_max = clean_data.max()
                                max_values_for_avg.append(interval_max)
                            else:
                                interval_max = 0
                                max_values_for_avg.append(0)
                            
                            # Add this max value once per interval
                            new_max_columns[new_col_name].append(interval_max)
                            
                            print(f"      Interval {interval+1}: rows {start_row+1}-{end_row} -> max = {interval_max}")
                        
                        # Handle remaining rows if any
                        if remaining_rows > 0:
                            start_row = 2 + (num_intervals * 500)
                            interval_data = df.iloc[start_row:][col]
                            
                            # Convert to numeric and handle non-numeric values
                            numeric_data = pd.to_numeric(interval_data, errors='coerce')
                            clean_data = numeric_data.dropna()
                            
                            if len(clean_data) > 0:
                                interval_max = clean_data.max()
                                max_values_for_avg.append(interval_max)
                            else:
                                interval_max = 0
                                max_values_for_avg.append(0)
                            
                            new_max_columns[new_col_name].append(interval_max)
                            
                            print(f"      Last interval: rows {start_row+1}-{len(df)} -> max = {interval_max}")
                        
                        # Calculate average of max values
                        if len(max_values_for_avg) > 0:
                            avg_max = np.mean(max_values_for_avg)
                            print(f"      Average of max values: {avg_max:.2f}")
                        else:
                            avg_max = 0
                        
                        # Store average for later use
                        new_max_columns[new_col_name + "_avg"] = avg_max
                    
                    print(f"  ✓ Completed processing all columns for this file")
                    
                    # Create the final dataframe structure
                    print(f"  Creating final dataframe structure...")
                    
                    # Start with original data
                    final_df = df.copy()
                    
                    # Add max columns with proper structure
                    for new_col_name, values in new_max_columns.items():
                        if not new_col_name.endswith("_avg"):  # Skip average values for now
                            # Create a column with the same length as original data
                            full_column = [""] * len(df)
                            
                            # Fill in the values we have (headers + max values)
                            for i, value in enumerate(values):
                                if i < len(full_column):
                                    full_column[i] = value
                            
                            final_df[new_col_name] = full_column
                    
                    # Add 4 empty rows after the last data row
                    empty_rows = 4
                    for i in range(empty_rows):
                        new_row = {col: "" for col in final_df.columns}
                        final_df = pd.concat([final_df, pd.DataFrame([new_row])], ignore_index=True)
                    
                    # Add the average values row
                    avg_row = {}
                    for col in final_df.columns:
                        avg_key = col + "_avg"
                        if avg_key in new_max_columns:
                            avg_row[col] = new_max_columns[avg_key]
                        else:
                            # For original columns, put "Average" label in the first column, empty in others
                            if col == final_df.columns[0]:
                                avg_row[col] = "Average of Max Values"
                            else:
                                avg_row[col] = ""
                    
                    final_df = pd.concat([final_df, pd.DataFrame([avg_row])], ignore_index=True)
                    
                    print(f"  Final dataframe shape: {final_df.shape}")
                    
                    # Save the processed file as Excel
                    file_name = os.path.splitext(os.path.basename(file_path))[0]
                    output_path = os.path.join(directory_path, f"{file_name}_processed.xlsx")
                    
                    final_df.to_excel(output_path, index=False)
                    
                    print(f"  ✓ File saved: {os.path.basename(output_path)}")
                    
                    # Count new columns added
                    max_cols = [col for col in final_df.columns if col.endswith('_Max500')]
                    print(f"  ✓ Added {len(max_cols)} max value columns")
                    print(f"  ✓ Added average row at the bottom")
                    
                except Exception as e:
                    print(f"  Error processing data for this file: {e}")
                    continue
                
            except Exception as e:
                print(f"Error reading file {os.path.basename(file_path)}: {e}")
                continue
        
        print(f"\n{'='*80}")
        print("BATCH PROCESSING COMPLETED")
        print(f"{'='*80}")
        print(f"Processed {len(csv_files)} CSV files")
        print(f"All processed Excel files saved in: {directory_path}")
        print("Look for files ending with '_processed.xlsx'")
        print("\nEach processed Excel file contains:")
        print("  - Original data columns from CSV")
        print("  - New max value columns (one max per 500 rows)")
        print("  - Average of max values at the bottom")

Processing all Excel files in directory...
Directory: C:\Users\thaim\Downloads\בדיקת פלאש\20-08\ניסוי 2\הזזתי בטעות
Found 2 CSV files:
  - בלי פילטר מלא - 12מ.csv
  - עם פילטר מלא - 12מ.csv

Starting to process CSV files...

Processing file 1/2: בלי פילטר מלא - 12מ.csv
File loaded successfully!
Original shape: (12160, 4)
Columns: ['Sensor S1 Signal', 'Sensor R2 Signal', 'Sensor R1 Signal', 'Sensor R2 Signal.1']
File structure detected:
  Row 1: [18458, 1563, 23403, 17596]
  Row 2: [18471, 1606, 23451, 17595]
  Data starts from row 3
  Total data rows: 12158
  Data analysis:
    Total data rows: 12158
    Complete 500-row intervals: 24
    Remaining rows in last interval: 158
    Total intervals: 25
    Processing column: Sensor S1 Signal
      Interval 1: rows 3-502 -> max = 5390911
      Interval 2: rows 503-1002 -> max = 5074023
      Interval 3: rows 1003-1502 -> max = 5856143
      Interval 4: rows 1503-2002 -> max = 5390684
      Interval 5: rows 2003-2502 -> max = 5500644
      I